In [2]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import xgboost as xgb
import datetime
from sklearn.linear_model import LassoLars

pd.set_option('display.float_format',lambda x: '%.5f'%x)

print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
#dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close()

f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfTrain = dfTrain.set_index(dfTrain['ID'])
dfPred = dfPred.rename(columns=col_name)
dfPred = dfPred.set_index(dfPred['ID'])
print('Y' in raw_predictors)

Load data...
Done.
False


In [3]:
Feat_file = ['Feat_cnt_col_non_missing','Feat_cnt_tool_non_missing','Feat_pcent_col_non_missing','Feat_pcent_tool_non_missing','Feat_minus_non_missing']
ex_feat = []
for filename in Feat_file:
    if filename == 'Feat_minus_non_missing':
        tmp = pd.read_csv('../../../'+filename+'.csv')
    else:
        tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')
    
predictors = raw_predictors+ex_feat

In [4]:
#define the input variables, dummies
predictors = raw_predictors+ex_feat
feat_cnt = 0
for key,value in var_change.items():
    feat_cnt+=1
    if feat_cnt>1:
        continue
    for var in var_change[key]['constant']:
        try:
            predictors.remove(var)
        except:
            continue
    for var in var_change[key]['category']:
        if not 'TOOL' in var:
            continue
        try:
            predictors.remove(var)
        except:
            continue
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        print(var)
        dfPred[var] = 0

TOOL_310_E
TOOL_340_2409.0
TOOL_344_T
TOOL_360_C


In [5]:
print(dfTrain[predictors].shape)
print(dfPred[predictors].shape)
print(len(predictors))

(500, 30458)
(100, 30458)
30458


In [14]:
##feature selection by lasso/lars
reg = LassoLars(alpha=0.01,copy_X=True, eps=..., fit_intercept=True,
     fit_path=True, max_iter=5000, normalize=True, positive=False,
     precompute='auto', verbose=False)
reg.fit( dfTrain[predictors], dfTrain['Y'])
print(reg.coef_)

[ 0.  0.  0. ...,  0.  0.  0.]


In [42]:
##feature selection by RF
from sklearn.ensemble import RandomForestRegressor  
import numpy as np  

'''rf = RandomForestRegressor(n_estimators=5000, max_features='sqrt',  max_depth=4, random_state=615)  
rf.fit(dfTrain[predictors], dfTrain['Y'])  

rf_imp_df = pd.DataFrame({'var':predictors,'imp':rf.feature_importances_})
rf_imp_df.sort_values('imp',ascending=False,inplace =True)
'''
feat_num = 2000
creteria = rf_imp_df.iloc[feat_num-1,0]
predictors_rf = rf_imp_df.loc[rf_imp_df['imp']>=creteria,'var'].values.tolist()
print(len(predictors_rf))

2000


In [43]:
##feature selection by pearson correlation
'''corr = dfTrain[predictors+['Y']].corr()
corr_y = corr.loc[corr['Y']!=1&corr['Y'].notnull(),'Y']'''
for i in range(20000):
    criteria = 0.3 - 0.0001*i
    length = (corr_y<-criteria).sum()+(corr_y>criteria).sum()
    if length>=2000:
        ind = corr.loc[(corr['Y']<-criteria)|(corr['Y']>criteria),'Y']
        break
ind = np.abs(ind)
ind = ind.sort_values(ascending=False)
predictors_pear = ind.iloc[0:len(predictors_rf)+1].index.tolist()
predictors_pear.remove('Y')

In [44]:
print(len(predictors_pear))

2000


In [45]:
selected_feat_df = pd.DataFrame({'RandomForest':predictors_rf,'PearsonCorr':predictors_pear})


In [46]:
selected_feat = []
i=0
while len(selected_feat)<2000:
    if selected_feat_df.loc[i,'RandomForest'] not in selected_feat:
        selected_feat.append(selected_feat_df.loc[i,'RandomForest'])
    if selected_feat_df.loc[i,'PearsonCorr'] not in selected_feat:
        selected_feat.append(selected_feat_df.loc[i,'PearsonCorr'])
    i+=1
print(i)
print(len(selected_feat))

1279
2000


In [47]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5,weight = 1,early_stop = 10 ,params = {'max_depth':3, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,
                         'colsample_bytree':0.8}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    if weight>1:
        tmp = np.abs(dfTrain['Y'] - dfTrain['Y'].quantile(0.5))
        dfTrain['wgt'] = 1+(weight-1)*(np.abs(dfTrain['Y'] - dfTrain['Y'].quantile(0.5))/np.abs(dfTrain['Y'] - dfTrain['Y'].quantile(0.5)).max())
    else:
        dfTrain['wgt'] = 1
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']
        train_wgt = dfTrain.loc[train_index,'wgt']
        test_wgt = dfTrain.loc[test_index,'wgt']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values,weight=train_wgt, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values,weight=test_wgt, missing = np.nan)
        param = params 
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=early_stop,evals_result=evals_dict,verbose_eval =100)
        performance_df = pd.DataFrame(evals_dict['eval'])
        #print(performance_df)
        bst_tree = len(performance_df)-1-early_stop
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp

In [50]:
n_splits =10
test_result,result,imp = xgb_kfold(dfTrain,dfPred,selected_feat,n_splits=n_splits,early_stop=5)

[0]	train-rmse:2.32957	eval-rmse:2.34906
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 5 rounds.
[100]	train-rmse:0.874241	eval-rmse:0.888242
[200]	train-rmse:0.34924	eval-rmse:0.363959
[300]	train-rmse:0.166922	eval-rmse:0.203916
[400]	train-rmse:0.108388	eval-rmse:0.170549
[500]	train-rmse:0.087621	eval-rmse:0.165389
Stopping. Best iteration:
[496]	train-rmse:0.088133	eval-rmse:0.165365

[0]	train-rmse:2.33307	eval-rmse:2.31658
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 5 rounds.
[100]	train-rmse:0.875347	eval-rmse:0.864241
[200]	train-rmse:0.350018	eval-rmse:0.355273
[300]	train-rmse:0.168015	eval-rmse:0.201916
[400]	train-rmse:0.110297	eval-rmse:0.168314
[500]	train-rmse:0.089075	eval-rmse:0.161177
Stopping. Best iteration:
[520]	train-rmse:0.086411	eval-rmse:0.160283

[0]	train-rmse:2.32874	eval-rmse:2.3558
M

In [51]:
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)
test_result.to_csv('../../Submission/test/test_result_%s.csv'%today,index=False)
imp.to_csv('../../Submission/imp/importance_%s.csv'%today,index=False)

In [ ]:
#RF & Pearson
1000  0.0264600389016

In [ ]:
#RF
500   0.0261706354963
800   0.0261447242005
1000  0.025398174979 / 0.026301476152
2000  0.0259095708883

In [ ]:
5000   0.0275283319973
3000   0.0276070372856
2000   0.0273172188618
1000   0.0275733258446
500    0.0272536288561
